# Подготовка изображеий для детектирования

## Установка и импорт необходимых библиотек

In [27]:
!pip install opencv-python
!pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=859deb9c47e217294cbce5bce74a26aae5c5034cda401087331ce2d060643d59
  Stored in directory: c:\users\alexa\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [24]:
# Import opencv - Для работы с изображениями
import cv2 
# Import uuid - Для генерации названий изображений
import uuid
# Import Operating System - Для работы с файловой системой
import os
# Import time - Для отслеживания времени
import time
#Import random - для разделение на тренировочную и тестовую выборки
import numpy

## 2. Определим классы изображений


In [4]:
# Так как задача - детекция номеров автомобилей, 
#список labels будет содержать один элемент 'number'
labels = ['number']
number_imgs = 2

## 3. Создадим каталоги для организации проекта

In [5]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [6]:
if not os.path.exists(IMAGES_PATH):
    # При запуске проекта на unix системах
    if os.name == 'posix':
        !mkdir -p {IMAGES_PATH}
    # При запуске проекта на Windows
    if os.name == 'nt':
         !mkdir {IMAGES_PATH}
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

## 4. Изображение были скачены с интернета с использованием Google изображений и Yandex картинок

## 5. Разметка изображений с использованием LabelImg

In [7]:
# установка необходимых библиотек
!pip install --upgrade pyqt5 lxml

In [8]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

In [13]:
# Клонируем LabelImg, если еще этого не делали
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

Cloning into 'Tensorflow\labelimg'...


In [15]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [18]:
# Запуск LAbelImg
!cd {LABELIMG_PATH} && python labelImg.py

## 6. Разделение данных на тренировачную и тестовую выборку

In [28]:
from sklearn.model_selection import train_test_split

In [ ]:
path = os.path.join(IMAGES_PATH, label)
PATH_TRAIN = os.path.join(path,"train")
PATH_TEST = os.path.join(path,"test")
list_files = []
for file in os.listdir(path):
    if ".jpg" in file:
        sample = (file,file[:-3]+"xml")
        list_files.append(sample)
x_train, x_test = train_test_split(list_files, test_size=0.2, random_state=11)
for image, xml in x_train:
    if os.name == "posix":
        if not os.path.exists(PATH_TRAIN):
            !mkdir {PATH_TRAIN}
        !mv {os.path.join(path,image)} {os.path.join(PATH_TRAIN,image)}
        !mv {os.path.join(path,xml)} {os.path.join(PATH_TRAIN,xml)}
    if os.name == "nt":
        if not os.path.exists(PATH_TRAIN):
            !mkdir {PATH_TRAIN}
        !move {os.path.join(path,image)} {os.path.join(PATH_TRAIN,image)}
        !move {os.path.join(path,xml)} {os.path.join(PATH_TRAIN,xml)}
for image, xml in x_test:
    if os.name == "posix":
        if not os.path.exists(PATH_TEST):
            !mkdir {PATH_TEST}
        !mv {os.path.join(path,image)} {os.path.join(PATH_TEST,image)}
        !mv {os.path.join(path,xml)} {os.path.join(PATH_TEST,xml)}
    if os.name == "nt":
        if not os.path.exists(PATH_TEST):
            !mkdir {PATH_TEST}
        !move {os.path.join(path,image)} {os.path.join(PATH_TEST,image)}
        !move {os.path.join(path,xml)} {os.path.join(PATH_TEST,xml)}

## (Дополнительный модуль). Получить Изображения можно с web-камеры при необходимости. 

In [ ]:
for label in labels:
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()